# What is Selenium?

## 1

What environment do we use?

See [What is Docker?](What%20is%20Docker.ipynb)

Run Selenium in Docker?


In [1]:
var corePath = path.resolve(path.join(__dirname, '..', 'Core'));
var importer = require(corePath);
var path = require('path');

// add some run commands to the bash script
var bashToRun = (code) => code.split('\n').map(l => 'RUN ' + l)
    .join('\n').replace(/\\\s*\nRUN\s*/ig, '\\\n ');

// create a selenium Dockerfile with a vnc connection
var interpretAllPath = path.join('..', 'Core', 'How to interpret all notebooks.ipynb');
var interpretPath = path.join('..', 'Core', 'How to interpret questions.ipynb');
var seleniumDocker = () => importer.importNotebook(interpretAllPath)
    //.then(() => importer.importNotebook(interpretPath))
    //.then(() => Promise.all([
    //    interpret('run selenium'),
    //  add some extra services
    //    interpret('vnc html'),
    //    interpret('vnc docker')
    //]))
    // lets fix the middle result
    //.then(r => interpretObject(r))
    //.then(r => {
    //    r[1][0].code = bashToRun(r[1][0].code);
    //    return r.map(r => r[0].code).join('\n');
    //});
    // TODO: save the Dockerfile
//$$.sendResult(seleniumDocker);
$$.async()
seleniumDocker()
    .then(r => $$.sendResult(r))
    .catch(r => $$.sendError(r));

[ { id: 'How to test import.ipynb[1]',
    question: 'how to test for errors line 2?' } ]


[ { id: 'How to test import.ipynb[1]',
    question: 'how to test for errors line 2?' },
  { id: 'How to import a notebook-checkpoint.ipynb[0]',
    question: '# How to execute and import a jupyter notebook?\n\n' },
  { id: 'How to import a notebook-checkpoint.ipynb[0]',
    question: '\nHow to use inheritence in jupyter notebooks?\n\n' },
  { id: 'How to import a notebook-checkpoint.ipynb[0]',
    question: '\nHow to use jupyter notebooks as services?' },
  { id: 'How to import a notebook-checkpoint.ipynb[1]',
    question: 'What do I need to have installed?\n\n' },
  { id: 'How to import a notebook-checkpoint.ipynb[2]',
    question: 'How to stream json using a {match} function?' },
  { id: 'How to import a notebook-checkpoint.ipynb[3]',
    question: 'How to parse cells of {types} from a notebook?' },
  { id: 'How to import a notebook-checkpoint.ipynb[4]',
    question: 'How to run all promis

{ 'How to interpret all notebooks.ipynb[0]': [Function],
  walkSync: [Function],
  'How to interpret all notebooks.ipynb[1]': [Function],
  interpretAll: [Function] }

In [ ]:
seleniumDocker()

# 2

How to connect to VNC through the web browser?

[Click here to open](http://localhost:6080/vnc.html?password=secret&host=localhost&port=6080&autoconnect=true&resize=downscale&view_only=true) in a separate browser


In [ ]:
var vncIframe = () => `
<div style="display:block; width:100%; padding-bottom:75%;position:relative;">
<iframe id="vnc"
style="position:absolute;top:0;right:0;bottom:0;left:0;width:100%;height:100%;border:0;"
src="http://localhost:6080/vnc.html?password=secret&amp;host=localhost&amp;port=6080&amp;autoconnect=true&amp;resize=downscale&amp;view_only=true"></iframe></div>
`;
(vncIframe);

## 3

How to run a selenium test on the Docker machine?
